Comparing bonus unemployment, payroll tax cuts, and universal payments
=============================================================

This uses the 2018 Current Population Survey March Supplement, Tax-Calculator, and the Supplemental Poverty Measure to estimate the effects of the Federal Pandemic Unemployment Compensation (extra 600 dollars per week), a budget-neutral payroll tax cuts, and a budget-neutral universal payment.

In [1]:
import numpy as np
import pandas as pd
import microdf as mdf
import plotly.express as px
import plotly

spmu = pd.read_csv('spmu.csv.gz')
person = pd.read_csv('person.csv.gz')

# All potential reforms.
CHG_COLS = ['fpuc_net', 'fpuc_ubi', 'fpuc_adult_ubi', 'fpuc_fica_cut',
            'fpuc2_net', 'fpuc2_ubi', 'fpuc2_adult_ubi', 'fpuc2_fica_cut']

## Poverty analysis

In [2]:
def pov(reform, age_group='All', race='All'):
    """ Calculate the poverty rate under the specified reform for the
        specified population.
        Note: All arguments refer to the poverty population, not the reform.
    
    Args:
        reform: One of CHG_COLS. If None, provides the baseline rate.
        age_group: Age group, either
            - 'Children' (under 18)
            - 'Adults' (18 or over)
            - 'All'
        race: Race code to filter to. Defaults to 'All'.
        
    Returns:
        2018 SPM poverty rate.
    """
    if reform == 'baseline':
        resource_col = 'spmpoor'
    else:
        resource_col = 'spmpoor_' + reform
    target_persons = person
    if age_group == 'Children':
        target_persons = target_persons[target_persons.age < 18]
    elif age_group == 'Adults':
        target_persons = target_persons[target_persons.age >= 18]
    if race != 'All':
        target_persons = target_persons[target_persons.race == race]
    return mdf.weighted_mean(target_persons, resource_col, 'asecwt')

def pov_row(row):
    """ Calculate poverty based on parameters specified in the row.
    
    Args:
        row: pandas Series.
        
    Returns:
        2018 SPM poverty rate.
    """
    return pov(row.reform, row.age_group, row.race)

In [3]:
pov_rates = mdf.cartesian_product({'reform': ['baseline'] + CHG_COLS,
                                   'age_group': ['All', 'Children', 'Adults'],
                                   'race': ['All', 200]})  # 200 means Black.
pov_rates['pov'] = 100 * pov_rates.apply(pov_row, axis=1)

Top-level poverty rates.

In [4]:
pov_rates[(pov_rates.age_group == 'All') & (pov_rates.race == 'All')]

,reform,age_group,race,pov
0,baseline,All,All,12.735766
6,fpuc_net,All,All,11.947370
12,fpuc_ubi,All,All,12.049012
18,fpuc_adult_ubi,All,All,12.143941
24,fpuc_fica_cut,All,All,12.552390
30,fpuc2_net,All,All,11.787156
36,fpuc2_ubi,All,All,11.652867
42,fpuc2_adult_ubi,All,All,11.699642
48,fpuc2_fica_cut,All,All,11.858684


Top-level child poverty rates.

In [5]:
pov_rates[(pov_rates.age_group == 'Children') & (pov_rates.race == 'All')]

,reform,age_group,race,pov
2,baseline,Children,All,13.657287
8,fpuc_net,Children,All,12.630711
14,fpuc_ubi,Children,All,12.648166
20,fpuc_adult_ubi,Children,All,13.119569
26,fpuc_fica_cut,Children,All,13.411898
32,fpuc2_net,Children,All,12.405195
38,fpuc2_ubi,Children,All,12.217862
44,fpuc2_adult_ubi,Children,All,12.404361
50,fpuc2_fica_cut,Children,All,12.503603


### Poverty gap

In [6]:
def pov_gap_b(reform):
    if reform == 'baseline':
        resource_col = 'spmtotres'
    else:
        resource_col = 'spmtotres_' + reform
    pov_gap = np.maximum(spmu.spmthresh - spmu[resource_col], 0)
    return (pov_gap * spmu.spmwt).sum() / 1e9

In [7]:
pov_gap = pd.DataFrame({'reform': ['baseline'] + CHG_COLS})
pov_gap['pov_gap_b'] = pov_gap.reform.apply(pov_gap_b)
pov_gap

,reform,pov_gap_b
0,baseline,169.993787
1,fpuc_net,161.600128
2,fpuc_ubi,161.393132
3,fpuc_adult_ubi,161.512723
4,fpuc_fica_cut,168.571358
5,fpuc2_net,160.547785
6,fpuc2_ubi,157.949613
7,fpuc2_adult_ubi,157.985658
8,fpuc2_fica_cut,161.016850


### Inequality

By individual based on their percentage of SPM resources.

In [8]:
def gini(reform):
    if reform == 'baseline':
        resource_col = 'spmtotres'
    else:
        resource_col = 'spmtotres_' + reform
    return mdf.gini(person[resource_col], person.asecwt)

In [9]:
pov_gap['gini'] = pov_gap.reform.apply(gini)
pov_gap

,reform,pov_gap_b,gini
0,baseline,169.993787,0.431619
1,fpuc_net,161.600128,0.428153
2,fpuc_ubi,161.393132,0.428481
3,fpuc_adult_ubi,161.512723,0.428829
4,fpuc_fica_cut,168.571358,0.431641
5,fpuc2_net,160.547785,0.427069
6,fpuc2_ubi,157.949613,0.426778
7,fpuc2_adult_ubi,157.985658,0.426932
8,fpuc2_fica_cut,161.016850,0.428174
